In [1]:
# import prepare
import wrangle
import acquire
# import explore
# import model
# import curie_model
# explore.set_plotting_defaults()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import os
import time
import unicodedata
import re
import json
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from env import github_token, github_username

In [23]:
def make_urls ():
    # get url list
    url_list = wrangle.github_ac_urls()

    # Set base_url that will be used in get request
    base_url = "https://github.com"
        
    # List of full url needed to get readme info
    readme_url_list = []
    for url in url_list:
        full_url = base_url + url
        readme_url_list.append(full_url)
    return readme_url_list

In [24]:
url= make_urls()

# Acquire

In [25]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username



headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
            f"url: {url}"
        )
    return response_data

In [26]:
response_data = github_api_request(url)

InvalidSchema: No connection adapters were found for '['https://github.com/louis70109/animal-crossing-bot', 'https://github.com/KevinZjYang/AnimalCrossing', 'https://github.com/DrLer0/AnimalCrossingSocial', 'https://github.com/facelessloser/animal_crossing_automation', 'https://github.com/leaf7th/animal_crossing', 'https://github.com/Weava/daisy-mae-bot', 'https://github.com/Azurilex/Blathers', 'https://github.com/Universal-Team/LeafEdit', 'https://github.com/KingLycosa/acnhpoker_linux', 'https://github.com/jonathanfeng/new_horizons', 'https://github.com/heriet/AnimalCrossingQR', 'https://github.com/jamchamb/ac-editor-stringtables', 'https://github.com/chendaniely/animal_crossing', 'https://github.com/mutoo/aseprite-animalcrossing-qrcode-gen', 'https://github.com/mgaut72/animal-crossing-new-horizons-data', 'https://github.com/Moppler/Critterpedia', 'https://github.com/FIX94/ac-exploit-gc', 'https://github.com/wchill/ACNHAutoCataloger', 'https://github.com/MariuszDabrowski/ac-miles', 'h

In [5]:
def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )

In [6]:
def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

NameError: name 'REPOS' is not defined

# Prep 

# Explore 

# Model

# Conclusion 